# Project Playground - Modular LangGraph Hybrid System

This notebook provides a playground for testing various aspects of the Modular LangGraph Hybrid System, starting with the Answer Agent.

## Setup and Imports

In [1]:
import sys
import os
from pathlib import Path
import json
from datetime import datetime
import uuid
from pprint import pprint

os.chdir('..') 

# Add the project root to the path
project_root = Path().cwd()
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")

# Import project modules using direct imports
try:
    # Core imports
    from src.core.config.agent_config_manager import AgentConfigManager
    from src.core.context_manager import SQLiteContextProvider
    from src.interfaces.core.state_schema import HybridSystemState
    from src.nodes.answer_agent import AnswerAgentNode
    from src.integrations.llm_providers import *
    
    print("✅ All modules loaded successfully using direct imports")
    
except Exception as e:
    print(f"❌ Import error: {e}")
    print("\n💡 Trying simpler approach...")
    
    try:
        # Fallback: Create minimal versions for testing
        class MockAgentConfigManager:
            def __init__(self, config_dir, environment='development'):
                self.config_dir = Path(config_dir)
                self.environment = environment
                print(f"Mock AgentConfigManager initialized with {config_dir}")
            
            def get_summary(self):
                return {"status": "mock", "config_dir": str(self.config_dir)}
                
            def get_agent_config(self, agent_name):
                return None
                
        AgentConfigManager = MockAgentConfigManager
        SQLiteContextProvider = None
        AnswerAgentNode = None
        HybridSystemState = dict  # Use dict as fallback
        
        print("⚠️ Using mock implementations - limited functionality available")
        
    except Exception as e2:
        print(f"❌ Fallback also failed: {e2}")
        print("\n🔧 Manual Setup Required:")
        print("1. Ensure you're running from the project root directory")
        print("2. Fix relative imports in the source files") 
        print("3. Or run: cd /workspace && python -m jupyter notebook")

Project root: /workspace
✅ All modules loaded successfully using direct imports


## Initialize Configuration Manager

In [2]:
# Initialize the configuration manager
config_dir = project_root / 'config'
config_manager = AgentConfigManager(config_dir, environment='development')

print("📋 Configuration Manager Summary:")
pprint(config_manager.get_summary())

📋 Configuration Manager Summary:
{'agent_names': ['evaluator_agent',
                 'human_interface',
                 'escalation_router',
                 'answer_agent'],
 'agents_loaded': 4,
 'config_directory': '/workspace/config',
 'config_files_structure': {'agents': True,
                            'environments': True,
                            'shared': True},
 'environment': 'development',
 'models_configured': 8,
 'providers_configured': 3,
 'system_name': 'Modular LangGraph Hybrid System',
 'system_version': '1.0.0',
 'thresholds': {'confidence_threshold': 0.6,
                'escalation_score': 5.0,
                'max_retries': 5,
                'response_time_limit': 60,
                'session_timeout': 3600}}


## Answer Agent Configuration Display

In [3]:
# Get the answer agent configuration
answer_agent_config = config_manager.get_agent_config('answer_agent')

if answer_agent_config:
    print("🤖 Answer Agent Configuration:")
    print(f"Name: {answer_agent_config.name}")
    print(f"Description: {answer_agent_config.description}")
    print(f"Type: {answer_agent_config.type}")
    print()
    
    print("🎯 Model Configuration:")
    print(f"Preferred Model: {answer_agent_config.get_preferred_model()}")
    print(f"Fallback Models: {answer_agent_config.get_fallback_models()}")
    print()
    
    print("⚙️ Settings:")
    pprint(answer_agent_config.settings)
    print()
    
    print("🧠 Behavior Configuration:")
    pprint(answer_agent_config.behavior)
    print()
    
    print("📈 Escalation Configuration:")
    pprint(answer_agent_config.escalation)
    print()
    
    print("💬 System Prompt:")
    print(f"'{answer_agent_config.get_prompt('system')}'")
    print()
    
    print("📝 Available Prompt Templates:")
    templates = answer_agent_config.prompts.get('templates', {})
    for template_name, template_content in templates.items():
        print(f"  {template_name}: {template_content[:100]}{'...' if len(template_content) > 100 else ''}")
else:
    print("❌ Answer agent configuration not found")

🤖 Answer Agent Configuration:
Name: answer_agent
Description: Primary response generation agent that provides direct answers to user queries
Type: llm_agent

🎯 Model Configuration:
Preferred Model: anthropic_general_budget
Fallback Models: ['local_general_budget', 'local_general_standard']

⚙️ Settings:
{'max_retries': 3,
 'max_tokens': 2000,
 'response_time_limit': 30,
 'temperature': 0.7,
 'timeout': 30}

🧠 Behavior Configuration:
{'context_integration': True,
 'personalization': True,
 'response_style': 'clear_and_professional'}

📈 Escalation Configuration:
{'confidence_threshold': 0.7,
 'enable_auto_escalation': True,
 'escalation_triggers': ['low_confidence',
                         'user_dissatisfaction',
                         'repeat_query']}

💬 System Prompt:
'You are a helpful AI assistant. Provide accurate, helpful responses to user queries.  Use context from previous conversations when available to provide more personalized assistance.
'

📝 Available Prompt Templates:
  

## Initialize Context Provider and Answer Agent

In [4]:
# Initialize context provider
context_provider = SQLiteContextProvider(str(project_root / 'hybrid_system.db'))

# Initialize answer agent
try:
    answer_agent = AnswerAgentNode(config_manager, context_provider)
    print("✅ Answer Agent initialized successfully")
    
    # Display LLM provider information if available
    if answer_agent.llm_provider:
        print(f"🔗 LLM Provider: {type(answer_agent.llm_provider).__name__}")
        if hasattr(answer_agent.llm_provider, 'model_config'):
            print(f"📊 Model Config: {answer_agent.llm_provider.model_config}")
    else:
        print("⚠️ LLM Provider not initialized (likely due to missing API keys or model files)")
        
except Exception as e:
    print(f"❌ Failed to initialize Answer Agent: {e}")
    answer_agent = None

✅ 19:08:16.175 [INFO    ] factory         | Attempting to create provider: anthropic_general_budget
✅ Creating LLM provider: anthropic_general_budget → claude-3-5-haiku-20241022 (anthropic)
✅ 19:08:16.177 [INFO    ] claude-3-5-haiku-20241022 | Initializing LLM provider | model_name=claude-3-5-haiku-20241022
✅ 19:08:16.181 [INFO    ] claude-3-5-haiku-20241022 | LLM provider initialized successfully | model_name=claude-3-5-haiku-20241022
✅ 19:08:16.183 [INFO    ] factory         | Successfully created provider: anthropic_general_budget
✅ 19:08:16.184 [INFO    ] answer_agent    | Answer Agent LLM provider initialized | operation=initialize_llm_provider model_name=claude-3-5-haiku-20241022
✅ Answer Agent initialized successfully
🔗 LLM Provider: LLMProvider
📊 Model Config: {'type': 'anthropic', 'model_name': 'claude-3-5-haiku-20241022', 'temperature': 0.7, 'max_tokens': 2000, 'description': 'Anthropic Claude 3.5 Haiku - fast and efficient'}


## Test Questions and Answer Generation

In [5]:
# Helper function to create test state
def create_test_state(query: str, user_id: str = "test_user", session_id: str = None) -> HybridSystemState:
    if session_id is None:
        session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    return {
        "query_id": str(uuid.uuid4()),
        "user_id": user_id,
        "session_id": session_id,
        "query": query,
        "timestamp": datetime.now().isoformat(),
        "messages": [],
        "context": {},
        "next_action": "answer"
    }

# Helper function to display answer results
def display_answer_result(query: str, result: HybridSystemState):
    print(f"\n{'='*60}")
    print(f"🙋 Question: {query}")
    print(f"{'='*60}")
    
    if 'ai_response' in result:
        print(f"🤖 Answer: {result['ai_response']}")
        print()
        
        if 'initial_assessment' in result:
            assessment = result['initial_assessment']
            print("📊 Assessment:")
            print(f"  Context Used: {assessment.get('context_used', 'N/A')}")
            print(f"  Confidence: {assessment.get('confidence', 'N/A')}")
            print(f"  Response Time: {assessment.get('response_time', 'N/A')}s")
            print(f"  Next Action: {result.get('next_action', 'N/A')}")
    else:
        print("❌ No AI response generated")
    
    print(f"{'='*60}\n")

print("✅ Helper functions defined")

✅ Helper functions defined


In [6]:
# Test questions to evaluate the answer agent
test_questions = [
    "What is artificial intelligence?",
    "How does machine learning work?",
    "Can you explain the difference between supervised and unsupervised learning?",
    "What are the benefits of using LangGraph for building AI workflows?",
    "How would you implement a simple chatbot?"
]

print("🧪 Testing Answer Agent with various questions...\n")

if answer_agent:
    session_id = f"test_session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n🔄 Test {i}/{len(test_questions)}")
        
        try:
            # Create test state
            test_state = create_test_state(question, session_id=session_id)
            
            # Generate answer
            result = answer_agent(test_state)
            
            # Display result
            display_answer_result(question, result)
            
        except Exception as e:
            print(f"❌ Error processing question '{question}': {e}")
            print()
else:
    print("❌ Answer agent not available - skipping tests")
    print("💡 This is likely due to missing LLM configuration or API keys")
    print("💡 Check your environment variables and model configuration")

🧪 Testing Answer Agent with various questions...


🔄 Test 1/5
✅ 19:08:21.316 [INFO    ] claude-3-5-haiku-20241022 | Model call: claude-3-5-haiku-20241022 - generate_response | model_name=claude-3-5-haiku-20241022 operation=generate_response

🙋 Question: What is artificial intelligence?
🤖 Answer: Artificial Intelligence (AI) is a branch of computer science focused on creating intelligent machines that can simulate human-like cognitive functions such as learning, problem-solving, perception, reasoning, and understanding natural language. AI systems use algorithms, machine learning, and large datasets to perform tasks that traditionally require human intelligence. There are different types of AI, including:

1. Narrow/Weak AI: Designed for specific tasks (like voice assistants or recommendation systems)
2. General AI: Theoretical systems with human-like cognitive abilities across multiple domains
3. Machine Learning: AI systems that can learn and improve from experience without being expl

## Context Testing - Follow-up Questions

In [7]:
# Test context integration with follow-up questions
print("🔗 Testing context integration with follow-up questions...\n")

if answer_agent:
    context_session_id = f"context_test_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    # First question to establish context
    first_question = "What is Python programming language?"
    print("🔄 Establishing context...")
    
    try:
        test_state = create_test_state(first_question, session_id=context_session_id)
        result1 = answer_agent(test_state)
        display_answer_result(first_question, result1)
        
        # Follow-up questions that should use context
        followup_questions = [
            "What are its main advantages?",
            "Can you give me some examples of what it's used for?",
            "How does it compare to Java?"
        ]
        
        for i, followup in enumerate(followup_questions, 1):
            print(f"\n🔄 Follow-up {i}/{len(followup_questions)}")
            
            test_state = create_test_state(followup, session_id=context_session_id)
            result = answer_agent(test_state)
            display_answer_result(followup, result)
            
    except Exception as e:
        print(f"❌ Error in context testing: {e}")
        
else:
    print("❌ Answer agent not available - skipping context tests")

🔗 Testing context integration with follow-up questions...

🔄 Establishing context...
✅ 19:08:51.225 [INFO    ] claude-3-5-haiku-20241022 | Model call: claude-3-5-haiku-20241022 - generate_response | model_name=claude-3-5-haiku-20241022 operation=generate_response

🙋 Question: What is Python programming language?
🤖 Answer: Python is a high-level, interpreted programming language known for its simplicity, readability, and versatility. Here are some key characteristics of Python:

1. Key Features:
- Easy to learn and read
- Supports multiple programming paradigms (object-oriented, functional, procedural)
- Dynamically typed
- Extensive standard library
- Cross-platform compatibility

2. Uses:
- Web development
- Data science and machine learning
- Scientific computing
- Artificial intelligence
- Automation and scripting
- Game development
- Desktop applications

3. Advantages:
- Clean and straightforward syntax
- Large and supportive community
- Numerous third-party libraries and framewor

## Configuration Experimentation

In [8]:
# Experiment with different configuration settings
print("🧪 Configuration Experimentation Section")
print()

# Display current agent settings
if answer_agent_config:
    print("🔧 Current Agent Settings:")
    print(f"  Temperature: {answer_agent_config.get_setting('temperature', 'Not set')}")
    print(f"  Max Tokens: {answer_agent_config.get_setting('max_tokens', 'Not set')}")
    print(f"  Response Time Limit: {answer_agent_config.get_setting('response_time_limit', 'Not set')}")
    print(f"  Context Integration: {answer_agent_config.get_setting('context_integration', 'Not set')}")
    print(f"  Confidence Threshold: {answer_agent_config.get_setting('confidence_threshold', 'Not set')}")
    print()
    
    print("🎛️ Available Configuration Options:")
    print("  - Modify temperature for creativity vs consistency")
    print("  - Adjust max_tokens for response length")
    print("  - Enable/disable context integration")
    print("  - Change confidence thresholds for escalation")
    print("  - Switch between different models")
    print()
    
    print("💡 To experiment with settings:")
    print("  1. Modify config/agents/answer_agent/config.yaml")
    print("  2. Call config_manager.reload() to apply changes")
    print("  3. Reinitialize the answer agent")
    
else:
    print("❌ Agent configuration not available")

🧪 Configuration Experimentation Section

🔧 Current Agent Settings:
  Temperature: 0.7
  Max Tokens: 2000
  Response Time Limit: 30
  Context Integration: True
  Confidence Threshold: 0.7

🎛️ Available Configuration Options:
  - Modify temperature for creativity vs consistency
  - Adjust max_tokens for response length
  - Enable/disable context integration
  - Change confidence thresholds for escalation
  - Switch between different models

💡 To experiment with settings:
  1. Modify config/agents/answer_agent/config.yaml
  2. Call config_manager.reload() to apply changes
  3. Reinitialize the answer agent


## System Status and Diagnostics

In [9]:
# System status and diagnostics
print("🔍 System Status and Diagnostics")
print()

# Configuration status
print("📋 Configuration Status:")
summary = config_manager.get_summary()
for key, value in summary.items():
    if key != 'thresholds':  # Skip detailed thresholds for cleaner output
        print(f"  {key}: {value}")
print()

# Available agents
print("🤖 Available Agents:")
agents = config_manager.get_available_agents()
for agent in agents:
    agent_config = config_manager.get_agent_config(agent)
    if agent_config:
        print(f"  {agent}: {agent_config.description}")
print()

# Context provider status
print("🗄️ Context Provider Status:")
try:
    # Test context provider
    test_summary = context_provider.get_context_summary("test_user", "test_session")
    print(f"  Database connection: ✅ Working")
    print(f"  Context entries for test user: {test_summary.get('entries_count', 0)}")
except Exception as e:
    print(f"  Database connection: ❌ Error - {e}")
print()

# LLM Provider status
print("🔗 LLM Provider Status:")
if answer_agent and answer_agent.llm_provider:
    print(f"  Provider: ✅ {type(answer_agent.llm_provider).__name__}")
    if hasattr(answer_agent.llm_provider, 'model_config'):
        model_config = answer_agent.llm_provider.model_config
        print(f"  Model: {getattr(model_config, 'name', 'Unknown')}")
        print(f"  Type: {getattr(model_config, 'type', 'Unknown')}")
else:
    print(f"  Provider: ❌ Not available")
    print(f"  Reason: Likely missing API keys or model files")

print("\n✅ Diagnostics complete!")

🔍 System Status and Diagnostics

📋 Configuration Status:
  config_directory: /workspace/config
  environment: development
  system_name: Modular LangGraph Hybrid System
  system_version: 1.0.0
  agents_loaded: 4
  agent_names: ['evaluator_agent', 'human_interface', 'escalation_router', 'answer_agent']
  models_configured: 8
  providers_configured: 3
  config_files_structure: {'shared': True, 'agents': True, 'environments': True}

🤖 Available Agents:
  evaluator_agent: Response quality evaluation and escalation decision specialist
  human_interface: Manages handoffs between AI and human agents with context preservation
  escalation_router: Routes escalated queries to appropriate human agents based on expertise and priority
  answer_agent: Primary response generation agent that provides direct answers to user queries

🗄️ Context Provider Status:
  Database connection: ✅ Working
  Context entries for test user: 0

🔗 LLM Provider Status:
  Provider: ✅ LLMProvider
  Model: Unknown
  Type: U

## Next Steps

This playground provides a foundation for testing the Answer Agent. Here are some next steps you can explore:

1. **Test Other Agents**: Load and test the EvaluatorAgent and EscalationRouter
2. **Workflow Testing**: Test the complete hybrid workflow with all agents
3. **Configuration Experiments**: Modify agent configurations and observe behavior changes
4. **Performance Testing**: Test with larger volumes of queries
5. **Context Analysis**: Examine how context affects response quality over time
6. **Model Comparison**: Test different LLM models and compare their performance

Use the cells above as templates for building more comprehensive tests!